In [ ]:
import pandas as pd
from tqdm import tqdm


In [ ]:
!git clone https://git.miem.hse.ru/gakirichenko1/vkgan.git

In [ ]:
!mkdir images
!mv archive.zip images


In [ ]:
!wget https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
!wget https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl  

--2022-10-20 11:35:05--  https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp37-cp37m-linux_x86_64.whl

Resolving download.pytorch.org (download.pytorch.org)... 99.86.38.106, 99.86.38.37, 99.86.38.96, ...

Connecting to download.pytorch.org (download.pytorch.org)|99.86.38.106|:443... connected.

HTTP request sent, awaiting response... 200 OK

Length: 2041342154 (1.9G) [binary/octet-stream]

Saving to: ‘torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl’



torch-1.9.0+cu111-c 100%[===================>]   1.90G  31.5MB/s    in 17s     



2022-10-20 11:35:22 (115 MB/s) - ‘torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl’ saved [2041342154/2041342154]



--2022-10-20 11:35:22--  https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl

Resolving download.pytorch.org (download.pytorch.org)... 18.65.229.70, 18.65.229.67, 18.65.229.14, ...

Connecting to download.pytorch.org (download.pytorch.org)|18.65.229.70|:443... connected.

HTTP request sent, awai

In [ ]:
!pip install *.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Processing ./torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl

Processing ./torchvision-0.10.0+cu111-cp37-cp37m-linux_x86_64.whl





  Attempting uninstall: torch

    Found existing installation: torch 1.12.1+cu113

    Uninstalling torch-1.12.1+cu113:

      Successfully uninstalled torch-1.12.1+cu113

  Attempting uninstall: torchvision

    Found existing installation: torchvision 0.13.1+cu113

    Uninstalling torchvision-0.13.1+cu113:

      Successfully uninstalled torchvision-0.13.1+cu113

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.9.0+cu111 which is incompatible.

torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0+cu111 which is incompatible.



In [ ]:
import torch
print("Pytorch version：")
print(torch.__version__)


Pytorch version：
1.11.0


In [ ]:
!unzip /content/images/archive.zip

In [ ]:
import os
import random
import glob
import shutil
import warnings

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR


In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn import Conv2d, BCEWithLogitsLoss, DataParallel, AvgPool2d, ModuleList, LeakyReLU, ConvTranspose2d, Embedding


In [ ]:
torch.cuda.is_available()

True

In [ ]:
class config:
  batch_size = 64 if torch.cuda.is_available() else 4
  latent_dim = 120

In [ ]:
import pandas as pd
df = pd.DataFrame(columns = ['path', 'label'])

In [ ]:
for dir in glob.glob('../input/pokemon-image-dataset/images/' + '*'):
  for files in glob.glob(f'{dir}/*'):
    df.loc[len(df)] = [files,dir.split('/')[1]]

In [ ]:
df.head(5)

,path,label
0,../input/pokemon-image-dataset/images/Whimsico...,input
1,../input/pokemon-image-dataset/images/Whimsico...,input
2,../input/pokemon-image-dataset/images/Forretre...,input
3,../input/pokemon-image-dataset/images/Forretre...,input
4,../input/pokemon-image-dataset/images/Forretre...,input


In [ ]:
class GANDataset(Dataset):
    def __init__(self, df, transform = None):
        self.df = df
        self.transform = transform
            
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
      image = self.df.loc[idx]
      img = Image.open(image.path)  
      if self.transform:
        img = self.transform(img)
      return img 
      

 

In [ ]:
from torchvision import transforms
train_augmentation = transforms.Compose([
        transforms.Resize(size = (128,128)),
        transforms.ToTensor()
    ])

In [ ]:
train_data = GANDataset(df,transform = train_augmentation)

In [ ]:
train_data[0].shape

torch.Size([3, 128, 128])

In [ ]:
train_loader = DataLoader(train_data, batch_size = config.batch_size)

In [ ]:
##################################
units=[32,64,128,256,512,1024]
padding=[2,2,2,2,2]
k_size=[5,5,5,5,5]#kernal size
strides=[2,2,2,2,2]
fs=[8,8]
image_size=[3,128,128]
small_image_size=[3,57,57]
#################################

In [ ]:
class G_net(nn.Module):
    def __init__(self,in_dim=config.latent_dim):
        super(G_net,self).__init__()
        self.fc1= nn.Sequential(
            nn.Linear(in_dim,units[4]),
            nn.ReLU(),
            nn.Linear(units[4],units[3]*fs[0]*fs[1]),
            nn.ReLU(),
            nn.BatchNorm1d(units[3]*fs[0]*fs[1])
        )
        self.ct1 = nn.Sequential(
            nn.ConvTranspose2d(units[3],units[2],k_size[3],stride=strides[3],padding=padding[3],output_padding=strides[3]//2),
            nn.BatchNorm2d(units[2]),
            nn.ReLU()
        )#[64,12,12]
        self.ct2 = nn.Sequential(
            nn.ConvTranspose2d(units[2],units[1],k_size[2],stride=strides[2],padding=padding[2],output_padding=strides[2]//2),
            nn.BatchNorm2d(units[1]),
            nn.ReLU()
        )#[32,27,27]
        self.ct3 = nn.Sequential(
            nn.ConvTranspose2d(units[1],units[0],k_size[1],stride=strides[1],padding=padding[1],output_padding=strides[1]//2),
            nn.BatchNorm2d(units[0]),
            nn.ReLU()
        )#[3,57,57]
        self.ct4 = nn.Sequential(
            nn.ConvTranspose2d(units[0],small_image_size[0],k_size[0],stride=strides[0],padding=padding[0],output_padding=strides[0]//2),
            nn.Tanh()
        )#[3,289,289]
    def forward(self,X):
        X = self.fc1(X)
        X = self.ct1(X.view(-1,units[3],fs[0],fs[1]))
        X = self.ct2(X)
        X = self.ct3(X)
        return self.ct4(X)

In [ ]:
class D_net(nn.Module):
    def __init__(self):
        super(D_net,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(image_size[0],units[0],k_size[0],strides[0],padding=padding[0]),
            nn.BatchNorm2d(units[0]),
            nn.LeakyReLU(0.2)
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(units[0],units[1],k_size[1],strides[1],padding=padding[1]),
            nn.BatchNorm2d(units[1]),
            nn.LeakyReLU(0.2)
        )
        self.conv3=nn.Sequential(
            nn.Conv2d(units[1],units[2],k_size[2],strides[2],padding=padding[2]),
            nn.BatchNorm2d(units[2]),
            nn.LeakyReLU(0.2)
        )
        self.conv4=nn.Sequential(
            nn.Conv2d(units[2],units[3],k_size[3],strides[3],padding=padding[3]),
            nn.BatchNorm2d(units[3]),
            nn.LeakyReLU(0.2)
        )
        self.fc1 = nn.Linear(units[3]*fs[0]*fs[1],units[4])
        self.dp = nn.Dropout(0.5)
        self.d_out = nn.Linear(units[4],1)
    def forward(self,X):
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.conv3(X)
        X = self.conv4(X)
        X = X.view((-1,units[3]*fs[0]*fs[1]))
        X = self.dp(F.leaky_relu(self.fc1(X)))
        out = self.d_out(X)
        return out

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
d_model = D_net().to(device)
g_model = G_net(config.latent_dim).to(device)
d_model.apply(weights_init)
g_model.apply(weights_init)

G_net(
  (fc1): Sequential(
    (0): Linear(in_features=120, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=16384, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(16384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ct1): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (ct2): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (ct3): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (

In [ ]:
#EMA WEIGHTS GENERATOR
###################################
eG = G_net(config.latent_dim)
eG.apply(weights_init)
#################################

G_net(
  (fc1): Sequential(
    (0): Linear(in_features=120, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=16384, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(16384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ct1): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (ct2): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (ct3): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (

In [ ]:
from torchvision.transforms import ToPILImage

In [ ]:
def get_noise(batch_size=config.batch_size):
    return Variable(torch.rand(batch_size,config.latent_dim))

In [ ]:
LAMBDA = 10
def loss_with_penalty(D,rdata,fdata):
    alpha = torch.rand(rdata.size())
    alpha = alpha.cuda(device) if use_GPU else alpha

    interpolates = alpha * rdata + ((1 - alpha) * fdata)

    interpolates = Variable(interpolates, requires_grad=True)
    if use_GPU:
        interpolates = interpolates.cuda(device)

    disc_interpolates = D(interpolates)
    gradients = grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda(device) if use_GPU else torch.ones(disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty


In [ ]:
from tqdm import tqdm

d_iter = 1
g_iter = 1
epoch = 3000
use_GPU = torch.cuda.is_available()
if use_GPU:
    device = torch.device('cuda')
from torch.autograd import Variable,grad
d_optimizer = Adam(d_model.parameters(),lr=1e-4,betas=[0.01,.9])
g_optimizer = Adam(g_model.parameters(),lr=1e-4,betas=[0.01,.9])
for e in range(epoch):
    with tqdm(train_loader, desc = 'Train', miniters=10) as progress:
        for data in progress:
            for i in range(d_iter):
                for p in d_model.parameters():
                    p.requires_grad = True
                d_model.zero_grad()
                true_data = Variable(data)

                if use_GPU:
                    true_data=true_data.cuda(device)
                d_true_score = d_model(true_data)
                true_loss = -d_true_score.mean()
                #fake data
                noise = get_noise(true_data.size()[0])
                if use_GPU:
                    noise = noise.cuda(device)
                fake_data = g_model(noise)
                d_fake_score = d_model(fake_data)
                fake_loss = d_fake_score.mean()
                w_loss = loss_with_penalty(d_model,true_data.data,fake_data.data)
                loss =true_loss+fake_loss+w_loss
                loss.backward()
                d_optimizer.step()
            for i in range(g_iter):
                g_model.zero_grad()
                for p in d_model.parameters():
                    p.requires_grad = False
                noise = get_noise()
                if use_GPU:
                    noise = noise.cuda(device)
                fake_data = g_model(noise)
                g_score = d_model(fake_data)
                g_loss = -g_score.mean()
                g_loss.backward()
                g_optimizer.step()
            progress.set_description(f'G loss: {g_loss.item() :.02f}, D_loss : {loss.item() :.02f}')
    if e%10==0:
        fake_imgs=fake_data.cpu()
        img = ToPILImage()(fake_imgs[3])
        img.save(f'./result/{e}.png')
        torch.save(d_model.state_dict(),'./result/d_model.pth')
        torch.save(g_model.state_dict(),'./result/generator.pth')


G loss: -7159.15, D_loss : -440.71:  28%|██▊       | 11/40 [00:06<00:16,  1.80it/s]


KeyboardInterrupt: 

In [ ]:
## generate file in repo (generate.py)

In [ ]:
def generate(count_img,save_path):
  g_model.eval()
  noise = get_noise(count_img)
  with torch.no_grad():
    generated = g_model(noise).cpu()
  for item in enumerate(generated):
        img = ToPILImage()(item[1])
        img.save(f'./{save_path}/{item[0]}.png')
